In [1]:
from pyflink.common import WatermarkStrategy, Row
from pyflink.common.typeinfo import Types
from pyflink.datastream import StreamExecutionEnvironment
from pyflink.datastream.connectors import NumberSequenceSource
from pyflink.datastream.functions import RuntimeContext, MapFunction
from pyflink.datastream.state import ValueStateDescriptor

In [2]:
#1. Create streamexecutionenvironment
env = StreamExecutionEnvironment.get_execution_environment()

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/opt/flink-1.15.0/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/opt/hadoop-2.7.7/share/hadoop/common/lib/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]


In [6]:

class MyMapFunction(MapFunction):

    def open(self, runtime_context: RuntimeContext):
        print("MyMapFunction open")
        state_desc = ValueStateDescriptor('cnt', Types.LONG())
        #Define value state
        self.cnt_state = runtime_context.get_state(state_desc)
        print ("Count state is ", self.cnt_state)
        

    def map(self, value):
        
        cnt = self.cnt_state.value()
        print("MyMapFunction map", cnt)
        if cnt is None:
            cnt = 0

        new_cnt = cnt + 1
        self.cnt_state.update(new_cnt)
        return value[0], new_cnt

In [3]:
#2. Create data source
seq_num_source = NumberSequenceSource(1, 10)

In [4]:
ds = env.from_source(
    source=seq_num_source,
    watermark_strategy=WatermarkStrategy.for_monotonous_timestamps(),
    source_name='seq_num_source',
    type_info=Types.LONG())

In [7]:

#3. Define execution logic
ds = ds.map(lambda a: Row(a % 4, 1), output_type=Types.ROW([Types.LONG(), Types.LONG()])) \
       .key_by(lambda a: a[0]) \
       .map(MyMapFunction(), output_type=Types.TUPLE([Types.LONG(), Types.LONG()]))


In [8]:
#4. Print the result data
ds.print()

In [9]:

#5. Perform the operation
env.execute()

MyMapFunction open
Count state is  <pyflink.fn_execution.state_impl.SynchronousValueRuntimeState object at 0x7f093aeccaf0>
MyMapFunction open
Count state is  <pyflink.fn_execution.state_impl.SynchronousValueRuntimeState object at 0x7f093aed2f70>
MyMapFunction open
Count state is  <pyflink.fn_execution.state_impl.SynchronousValueRuntimeState object at 0x7f093aeeb070>
MyMapFunction map None
MyMapFunction map 1
MyMapFunction map 2
MyMapFunction map None
MyMapFunction map None
MyMapFunction map 1
MyMapFunction map 2
MyMapFunction map 1
MyMapFunction map None
MyMapFunction map 1
2> (1,1)
2> (1,2)
2> (1,3)
3> (3,1)
3> (3,2)
1> (0,1)
1> (2,1)
1> (2,2)
1> (2,3)
1> (0,2)
